In [1]:
!pip install -q "tensorflow_io==0.28.*"
!pip install resampy
!pip install llvmlite==0.31.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for llvmlite
  Running setup.py clean for llvmlite
Failed to build llvmlite
ERROR: Could not build wheels for llvmlite, which is required to install pyproject.toml-based projects


In [21]:
import os
import time
import librosa
import zipfile
import numpy as np
import pandas as pd
import librosa.display
import IPython.display
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
import random
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_io as tfio
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

Downloading the dataset

In [ ]:
!wget https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz -O urban8k.tgz
!tar -xzf urban8k.tgz
!rm urban8k.tgz

In [4]:
data=pd.read_csv("UrbanSound8K/metadata/UrbanSound8K.csv")
data

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.000000,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.500000,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.500000,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.000000,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.500000,72.500000,1,5,2,children_playing
...,...,...,...,...,...,...,...,...
8727,99812-1-2-0.wav,99812,159.522205,163.522205,2,7,1,car_horn
8728,99812-1-3-0.wav,99812,181.142431,183.284976,2,7,1,car_horn
8729,99812-1-4-0.wav,99812,242.691902,246.197885,2,7,1,car_horn
8730,99812-1-5-0.wav,99812,253.209850,255.741948,2,7,1,car_horn


# Zero shot classification using pretrained models

## CLAP model

Loading the pre trained model

In [51]:
from transformers import AutoProcessor, ClapModel
from transformers import pipeline
model = ClapModel.from_pretrained("laion/clap-htsat-unfused")
processor = AutoProcessor.from_pretrained("laion/clap-htsat-unfused")

Defining the label set and a sample audio file for classification

In [63]:
labels = ['dog bark', 'children playing']
audio_sample, sr = librosa.load('UrbanSound8K/audio/fold5/100263-2-0-117.wav')
inputs = processor(text=labels, audios=audio_sample, return_tensors="pt", padding=True)
outputs = model(**inputs)

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [64]:
classifier = pipeline(
    task="zero-shot-audio-classification", model="laion/clap-htsat-unfused"
)
classifier(audio_sample, candidate_labels=labels)

[{'score': 0.9673612117767334, 'label': 'children playing'},
 {'score': 0.03263884410262108, 'label': 'dog bark'}]

We can see that the model is able to successfully distinguish between two labels, as the original class of the audio sample picked is 'children_playing'

Defining the entire label set from the dataset

In [68]:
labels = ['dog bark', 'children playing', 'air conditioner', 'car horn', 'drilling', 'engine idling', 'gun shot', 'jackhammer', 'siren', 'street music']
audio_sample, sr = librosa.load('UrbanSound8K/audio/fold5/100263-2-0-117.wav')
inputs = processor(text=labels, audios=audio_sample, return_tensors="pt", padding=True)
outputs = model(**inputs)

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [69]:
classifier = pipeline(
    task="zero-shot-audio-classification", model="laion/clap-htsat-unfused"
)
classifier(audio_sample, candidate_labels=labels)

[{'score': 0.6111621856689453, 'label': 'children playing'},
 {'score': 0.27071577310562134, 'label': 'street music'},
 {'score': 0.06318014860153198, 'label': 'siren'},
 {'score': 0.020620649680495262, 'label': 'dog bark'},
 {'score': 0.01248602569103241, 'label': 'gun shot'},
 {'score': 0.009232155047357082, 'label': 'car horn'},
 {'score': 0.007526170462369919, 'label': 'drilling'},
 {'score': 0.0023479561787098646, 'label': 'air conditioner'},
 {'score': 0.0019175066845491529, 'label': 'engine idling'},
 {'score': 0.0008114249212667346, 'label': 'jackhammer'}]

Wh

## YAMNET

Loading the pre trained model and displaying the classes it has been trained on

In [11]:
model_handle = 'https://tfhub.dev/google/yamnet/1'
model = hub.load(model_handle)
path = model.class_map_path().numpy().decode('utf-8')
classes =list(pd.read_csv(path)['display_name'])
for name in classes:
  print(name)


Speech
Child speech, kid speaking
Conversation
Narration, monologue
Babbling
Speech synthesizer
Shout
Bellow
Whoop
Yell
Children shouting
Screaming
Whispering
Laughter
Baby laughter
Giggle
Snicker
Belly laugh
Chuckle, chortle
Crying, sobbing
Baby cry, infant cry
Whimper
Wail, moan
Sigh
Singing
Choir
Yodeling
Chant
Mantra
Child singing
Synthetic singing
Rapping
Humming
Groan
Grunt
Whistling
Breathing
Wheeze
Snoring
Gasp
Pant
Snort
Cough
Throat clearing
Sneeze
Sniff
Run
Shuffle
Walk, footsteps
Chewing, mastication
Biting
Gargling
Stomach rumble
Burping, eructation
Hiccup
Fart
Hands
Finger snapping
Clapping
Heart sounds, heartbeat
Heart murmur
Cheering
Applause
Chatter
Crowd
Hubbub, speech noise, speech babble
Children playing
Animal
Domestic animals, pets
Dog
Bark
Yip
Howl
Bow-wow
Growling
Whimper (dog)
Cat
Purr
Meow
Hiss
Caterwaul
Livestock, farm animals, working animals
Horse
Clip-clop
Neigh, whinny
Cattle, bovinae
Moo
Cowbell
Pig
Oink
Goat
Bleat
Sheep
Fowl
Chicken, rooster
Cluck
Crowi

In [14]:
scores, embeddings, spectrogram = model(audio_sample)
scores = tf.reduce_mean(scores, axis=0)
top_class = tf.math.argmax(scores)
pred = classes[top_class]
print(f'predicted class: {pred}')

predicted class: Speech


# Training

In [17]:
import numpy as np
from tqdm import tqdm
import os
import resampy

def mfcc(file):
    audio, sample_rate = librosa.load(file_name)
    mfcc = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfcc_s = np.mean(mfcc.T,axis=0)
    return mfcc_s

dataset='UrbanSound8K/audio/'
metadata = pd.read_csv('UrbanSound8K/metadata/UrbanSound8K.csv')

In [19]:
features=[]
for index_num,row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(dataset),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    labels=row["class"]
    data=mfcc(file_name)
    features.append([data,labels])

3555it [03:56, 16.33it/s]/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1323
  warnings.warn(
8324it [06:53, 22.40it/s]/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1103
  warnings.warn(
8328it [06:53, 26.05it/s]/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1523
  warnings.warn(
8732it [07:08, 20.38it/s]


In [20]:
features_df=pd.DataFrame(features,columns=['feature','class'])
features_df.head()

,feature,class
0,"[-211.93698, 62.581215, -122.813156, -60.74529...",dog_bark
1,"[-417.0052, 99.336624, -42.995586, 51.073326, ...",children_playing
2,"[-452.39316, 112.36253, -37.578075, 43.195866,...",children_playing
3,"[-406.47922, 91.1966, -25.043556, 42.78452, 11...",children_playing
4,"[-439.63873, 103.86223, -42.658787, 50.690285,...",children_playing


In [24]:
X=np.array(features_df['feature'].tolist())
y=np.array(features_df['class'].tolist())
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [32]:
num_labels=10
model=Sequential()
model.add(Dense(200,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(300))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(300))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()


model.compile(loss='categorical_crossentropy',metrics=['accuracy',tf.keras.metrics.Recall(),tf.keras.metrics.Precision(), tf.keras.metrics.F1Score()],optimizer='adam')

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 200)               8200      
                                                                 
 activation_8 (Activation)   (None, 200)               0         
                                                                 
 dropout_6 (Dropout)         (None, 200)               0         
                                                                 
 dense_9 (Dense)             (None, 300)               60300     
                                                                 
 activation_9 (Activation)   (None, 300)               0         
                                                                 
 dropout_7 (Dropout)         (None, 300)               0         
                                                                 
 dense_10 (Dense)            (None, 300)              

In [33]:
checkpointer = ModelCheckpoint(filepath='models/classification.hdf5',
                               verbose=1, save_best_only=True)
start = datetime.now()
model.fit(X_train, y_train, batch_size=32, epochs=400, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)
print("time: ", datetime.now() - start)

Epoch 1/400
217/219 [============================>.] - ETA: 0s - loss: 8.8924 - accuracy: 0.1411 - recall_2: 0.1056 - precision_2: 0.1485 - f1_score: 0.1307
Epoch 1: val_loss improved from inf to 2.12885, saving model to models/classification.hdf5
219/219 [==============================] - 7s 18ms/step - loss: 8.8609 - accuracy: 0.1412 - recall_2: 0.1054 - precision_2: 0.1486 - f1_score: 0.1307 - val_loss: 2.1289 - val_accuracy: 0.3034 - val_recall_2: 0.0183 - val_precision_2: 1.0000 - val_f1_score: 0.2553
Epoch 2/400
 12/219 [>.............................] - ETA: 1s - loss: 3.0572 - accuracy: 0.1797 - recall_2: 0.0781 - precision_2: 0.1974 - f1_score: 0.1768

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


217/219 [============================>.] - ETA: 0s - loss: 2.5703 - accuracy: 0.1911 - recall_2: 0.0611 - precision_2: 0.2663 - f1_score: 0.1753
Epoch 2: val_loss improved from 2.12885 to 2.12486, saving model to models/classification.hdf5
219/219 [==============================] - 4s 16ms/step - loss: 2.5695 - accuracy: 0.1914 - recall_2: 0.0608 - precision_2: 0.2661 - f1_score: 0.1755 - val_loss: 2.1249 - val_accuracy: 0.2501 - val_recall_2: 0.0212 - val_precision_2: 0.9250 - val_f1_score: 0.2097
Epoch 3/400
219/219 [==============================] - ETA: 0s - loss: 2.2048 - accuracy: 0.2389 - recall_2: 0.0594 - precision_2: 0.4235 - f1_score: 0.2182
Epoch 3: val_loss improved from 2.12486 to 2.01150, saving model to models/classification.hdf5
219/219 [==============================] - 4s 18ms/step - loss: 2.2048 - accuracy: 0.2389 - recall_2: 0.0594 - precision_2: 0.4235 - f1_score: 0.2182 - val_loss: 2.0115 - val_accuracy: 0.3091 - val_recall_2: 0.0509 - val_precision_2: 0.9468 - v

In [39]:
test_accuracy= model.evaluate(X_test,y_test,verbose=0)
print('Test Accuracy:', test_accuracy[1], 'Test Loss:', test_accuracy[0], 'Recall:', test_accuracy[2], 'Precision:', test_accuracy[3], 'F1-Score:',test_accuracy[4])

Test Accuracy: 0.8729250431060791 Test Loss: 0.42425134778022766 Recall: 0.8139668107032776 Precision: 0.9367588758468628 F1-Score: [0.95431477 0.8915662  0.75784755 0.8042328  0.92583114 0.951049
 0.82666665 0.9358288  0.9457364  0.7229551 ]


In [40]:
num_labels=10
model=Sequential()
model.add(Dense(400,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(400))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(400))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()


model.compile(loss='categorical_crossentropy',metrics=['accuracy',tf.keras.metrics.Recall(),tf.keras.metrics.Precision(), tf.keras.metrics.F1Score()],optimizer='adam')

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 400)               16400     
                                                                 
 activation_12 (Activation)  (None, 400)               0         
                                                                 
 dropout_9 (Dropout)         (None, 400)               0         
                                                                 
 dense_13 (Dense)            (None, 400)               160400    
                                                                 
 activation_13 (Activation)  (None, 400)               0         
                                                                 
 dropout_10 (Dropout)        (None, 400)               0         
                                                                 
 dense_14 (Dense)            (None, 400)              

In [41]:
checkpointer = ModelCheckpoint(filepath='models/classification_v1.hdf5',
                               verbose=1, save_best_only=True)
start = datetime.now()
model.fit(X_train, y_train, batch_size=32, epochs=400, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)
print("time: ", datetime.now() - start)

test_accuracy= model.evaluate(X_test,y_test,verbose=0)
print('Test Accuracy:', test_accuracy[1], 'Test Loss:', test_accuracy[0], 'Recall:', test_accuracy[2], 'Precision:', test_accuracy[3], 'F1-Score:',test_accuracy[4])

Epoch 1/400
216/219 [============================>.] - ETA: 0s - loss: 6.4823 - accuracy: 0.1623 - recall_3: 0.0972 - precision_3: 0.1660 - f1_score: 0.1521
Epoch 1: val_loss improved from inf to 2.17742, saving model to models/classification_v1.hdf5
219/219 [==============================] - 5s 17ms/step - loss: 6.4430 - accuracy: 0.1631 - recall_3: 0.0969 - precision_3: 0.1665 - f1_score: 0.1529 - val_loss: 2.1774 - val_accuracy: 0.2038 - val_recall_3: 0.0172 - val_precision_3: 1.0000 - val_f1_score: 0.1849
Epoch 2/400
  4/219 [..............................] - ETA: 3s - loss: 2.2527 - accuracy: 0.2422 - recall_3: 0.0781 - precision_3: 0.3846 - f1_score: 0.2225

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


216/219 [============================>.] - ETA: 0s - loss: 2.2562 - accuracy: 0.2183 - recall_3: 0.0657 - precision_3: 0.4239 - f1_score: 0.2052
Epoch 2: val_loss improved from 2.17742 to 2.02911, saving model to models/classification_v1.hdf5
219/219 [==============================] - 4s 16ms/step - loss: 2.2565 - accuracy: 0.2175 - recall_3: 0.0656 - precision_3: 0.4237 - f1_score: 0.2046 - val_loss: 2.0291 - val_accuracy: 0.2570 - val_recall_3: 0.0550 - val_precision_3: 0.9412 - val_f1_score: 0.2659
Epoch 3/400
214/219 [============================>.] - ETA: 0s - loss: 2.0147 - accuracy: 0.2900 - recall_3: 0.1276 - precision_3: 0.6069 - f1_score: 0.2881
Epoch 3: val_loss improved from 2.02911 to 1.77533, saving model to models/classification_v1.hdf5
219/219 [==============================] - 2s 9ms/step - loss: 2.0125 - accuracy: 0.2903 - recall_3: 0.1276 - precision_3: 0.6074 - f1_score: 0.2885 - val_loss: 1.7753 - val_accuracy: 0.4322 - val_recall_3: 0.1099 - val_precision_3: 0.945

In [42]:
num_labels=10
model=Sequential()
model.add(Dense(600,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(600))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(600))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()


model.compile(loss='categorical_crossentropy',metrics=['accuracy',tf.keras.metrics.Recall(),tf.keras.metrics.Precision(), tf.keras.metrics.F1Score()],optimizer='adam')

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 600)               24600     
                                                                 
 activation_16 (Activation)  (None, 600)               0         
                                                                 
 dropout_12 (Dropout)        (None, 600)               0         
                                                                 
 dense_17 (Dense)            (None, 600)               360600    
                                                                 
 activation_17 (Activation)  (None, 600)               0         
                                                                 
 dropout_13 (Dropout)        (None, 600)               0         
                                                                 
 dense_18 (Dense)            (None, 600)              

In [43]:
checkpointer = ModelCheckpoint(filepath='models/classification_v2.hdf5',
                               verbose=1, save_best_only=True)
start = datetime.now()
model.fit(X_train, y_train, batch_size=32, epochs=400, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)
print("time: ", datetime.now() - start)

test_accuracy= model.evaluate(X_test,y_test,verbose=0)

Epoch 1/400
218/219 [============================>.] - ETA: 0s - loss: 5.7968 - accuracy: 0.1750 - recall_4: 0.0889 - precision_4: 0.1797 - f1_score: 0.1639
Epoch 1: val_loss improved from inf to 2.16770, saving model to models/classification_v2.hdf5
219/219 [==============================] - 8s 27ms/step - loss: 5.7921 - accuracy: 0.1754 - recall_4: 0.0888 - precision_4: 0.1797 - f1_score: 0.1642 - val_loss: 2.1677 - val_accuracy: 0.1935 - val_recall_4: 0.0321 - val_precision_4: 1.0000 - val_f1_score: 0.1600
Epoch 2/400
218/219 [============================>.] - ETA: 0s - loss: 2.1962 - accuracy: 0.2335 - recall_4: 0.0708 - precision_4: 0.4805 - f1_score: 0.2305
Epoch 2: val_loss improved from 2.16770 to 1.92509, saving model to models/classification_v2.hdf5
219/219 [==============================] - 6s 27ms/step - loss: 2.1973 - accuracy: 0.2332 - recall_4: 0.0707 - precision_4: 0.4796 - f1_score: 0.2302 - val_loss: 1.9251 - val_accuracy: 0.3234 - val_recall_4: 0.1002 - val_precision

In [45]:
num_labels=10
model=Sequential()
model.add(Dense(800,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(900))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(900))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()


model.compile(loss='categorical_crossentropy',metrics=['accuracy',tf.keras.metrics.Recall(),tf.keras.metrics.Precision(), tf.keras.metrics.F1Score()],optimizer='adam')

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 800)               32800     
                                                                 
 activation_20 (Activation)  (None, 800)               0         
                                                                 
 dropout_15 (Dropout)        (None, 800)               0         
                                                                 
 dense_21 (Dense)            (None, 900)               720900    
                                                                 
 activation_21 (Activation)  (None, 900)               0         
                                                                 
 dropout_16 (Dropout)        (None, 900)               0         
                                                                 
 dense_22 (Dense)            (None, 900)              

In [46]:
checkpointer = ModelCheckpoint(filepath='models/classification_v3.hdf5',
                               verbose=1, save_best_only=True)
start = datetime.now()
model.fit(X_train, y_train, batch_size=32, epochs=400, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)
print("time: ", datetime.now() - start)

test_accuracy= model.evaluate(X_test,y_test,verbose=0)

Epoch 1/400
218/219 [============================>.] - ETA: 0s - loss: 5.3857 - accuracy: 0.2117 - recall_5: 0.1221 - precision_5: 0.2370 - f1_score: 0.2005
Epoch 1: val_loss improved from inf to 2.00164, saving model to models/classification_v3.hdf5
219/219 [==============================] - 13s 50ms/step - loss: 5.3811 - accuracy: 0.2117 - recall_5: 0.1221 - precision_5: 0.2371 - f1_score: 0.2005 - val_loss: 2.0016 - val_accuracy: 0.3091 - val_recall_5: 0.0458 - val_precision_5: 0.8989 - val_f1_score: 0.3134
Epoch 2/400
218/219 [============================>.] - ETA: 0s - loss: 2.0223 - accuracy: 0.3131 - recall_5: 0.1451 - precision_5: 0.5349 - f1_score: 0.3178
Epoch 2: val_loss improved from 2.00164 to 1.74362, saving model to models/classification_v3.hdf5
219/219 [==============================] - 8s 38ms/step - loss: 2.0219 - accuracy: 0.3132 - recall_5: 0.1453 - precision_5: 0.5351 - f1_score: 0.3182 - val_loss: 1.7436 - val_accuracy: 0.3984 - val_recall_5: 0.1580 - val_precisio